In [1]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
import pandas as pd
import torch
import os


/Users/beratburakkaya/Documents/iot_timeseries_machine_learning_updated/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 🔧 Config
MODEL_NAME = "gpt2"  # Try also: "gpt2-medium", "TinyLlama", or "EleutherAI/gpt-neo-125M"
DATA_DIR = "../data/llm_preprocessed/"
OUTPUT_DIR = "../models/llm_forecaster/"


# ✅ Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# 📁 1. Dataset class using prompt + completion
class TimeSeriesPromptDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=256):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        prompt = self.data.iloc[idx]["prompt"]
        completion = self.data.iloc[idx]["completion"]
        full_text = prompt + " " + completion

        encodings = self.tokenizer(
            full_text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        input_ids = encodings["input_ids"].squeeze()
        attention_mask = encodings["attention_mask"].squeeze()
        labels = input_ids.clone()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

In [4]:
# 📚 2. Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  # Required for padding
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [5]:
# 📊 3. Prepare datasets using pre-split LLM files
train_dataset = TimeSeriesPromptDataset(f"{DATA_DIR}/train.csv", tokenizer)
val_dataset = TimeSeriesPromptDataset(f"{DATA_DIR}/val.csv", tokenizer)

In [6]:
# ⚙️ 4. Training Arguments (compatible with transformers 4.51.3 or lower)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    logging_dir=os.path.join(OUTPUT_DIR, "logs"),
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=500,
    logging_steps=500,
    weight_decay=0.01,
    save_total_limit=2,
    learning_rate=5e-5,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU
    report_to="none",  # disable W&B or Hub
)

In [7]:
# 🧠 5. Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/var/folders/s8/pp3cmgsj1h3gqdc1dpxr6yzm0000gn/T/ipykernel_16239/49609885.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# 🚀 6. Train and Save
trainer.train()
trainer.save_model(OUTPUT_DIR)
print(f"✅ LLM Fine-tuning complete. Model saved to {OUTPUT_DIR}")

/Users/beratburakkaya/Documents/iot_timeseries_machine_learning_updated/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.608400
1000,0.537900
1500,0.433900
2000,0.400400
2500,0.385300
3000,0.373900
3500,0.365300
4000,0.355000
4500,0.348600
5000,0.343200


/Users/beratburakkaya/Documents/iot_timeseries_machine_learning_updated/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/beratburakkaya/Documents/iot_timeseries_machine_learning_updated/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/beratburakkaya/Documents/iot_timeseries_machine_learning_updated/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/beratburakkaya/Documents/iot_timeseries_machine_learning_updated/.venv/lib/python3.13/site-packages/torch/utils/data/datalo

✅ LLM Fine-tuning complete. Model saved to ../models/llm_forecaster/
